In [3]:
pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ------------------ ------------------- 20.5/41.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 396.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 660.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.5 MB 919.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.5 MB 1.2 MB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.5 MB 1.4 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 1.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 1.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 1.4 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 846.5 kB/s eta 0:00:02
   ------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

In [5]:
# Download the VADER lexicon
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to C:\Users\Ishwari
[nltk_data]     Kadam\AppData\Roaming\nltk_data...


True

In [ ]:
#loading data from S3 bucket
s3_bucket = 'reviewssentimentanalysis'
file_name = 'RestaurantReviews.xlsx'

In [ ]:
# Use boto3 to download the file from S3
import boto3
s3 = boto3.client('s3')
s3.download_file(s3_bucket, file_name, 'RestaurantReviews.xlsx')

In [ ]:
df = pd.read_excel('RestaurantReviews.xlsx')

In [ ]:
# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
def get_sentiment(text):
    # Get polarity scores
    scores = sid.polarity_scores(text)
    compound = scores['compound']

    # Classify sentiment
    if compound >= 0.05:
        sentiment = 'Positive'
    elif compound <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    # Return scores along with sentiment label
    return compound, scores['pos'], scores['neu'], scores['neg'], sentiment

In [ ]:
df[['Compound', 'Positive', 'Neutral', 'Negative', 'Sentiment']] = df['Description'].apply(lambda x: pd.Series(get_sentiment(x)))

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
df[['Date', 'Group']] = df['DateandGroup'].str.split(' • ', expand=True)

In [ ]:
df.to_csv('sentiment_results.csv', index=False)

In [ ]:
# Upload the results back to S3
s3.upload_file('sentiment_results.csv', s3_bucket, 'sentiment_results.csv')

In [ ]:
print("Sentiment analysis completed and results saved to S3!")
